# 01 - Process and Merge VTAs
- Use if left and right VTAs are separate

In [36]:
from glob import glob
import subprocess
import os 
from nilearn.image import resample_to_img
import nibabel as nib
from tqdm import tqdm
import shutil 
from nimlab import datasets as nimds


from calvin_utils.nifti_utils.generate_nifti import nifti_from_matrix
from calvin_utils.file_utils.import_functions import GiiNiiFileImport


def downsample_image(input_path, output_path):
    """
    Function to downsample a 3D image to a new voxel size using a target affine.
    
    Args:
    input_path (str): Filepath to the input image.
    output_path (str): Filepath to save the output image.
    target_voxel_size (list): Target voxels to resample to.
    """
    # Load the image
    img = nib.load(input_path)
    mni_mask = nimds.get_img("mni_icbm152")
    
    # Downsample the image using the target affine
    resampled_img = resample_to_img(img, mni_mask)

    # Save the downsampled image
    nib.save(resampled_img, output_path)
    
def prepare_backup_files(file_in):
    # Prepare backup file name
    copyfile_name = file_in.split('.nii')[0] + "_copy.nii"
    # Copy files to create backups
    shutil.copyfile(file_in, copyfile_name)
    return copyfile_name
    
    
def warp_nifti(copyfile, mni_template_path):
    # Attempt resampling to MNI space to fix bad headers
    subprocess.run(["flirt", "-in", copyfile, "-ref", mni_template_path, "-out", copyfile], check=True)
            

def merge_files(root_dir, file_pattern_1, file_pattern_2, output_file_pattern, mni_template_path=None, dry_run=True):
    for sub_dir in tqdm(os.listdir(root_dir)):
        sub_dir_path = os.path.join(root_dir, sub_dir)
        if not os.path.isdir(sub_dir_path):
            continue            
        files1 = glob(root_dir+"/"+sub_dir+file_pattern_1)
        files2 = glob(root_dir+"/"+sub_dir+file_pattern_2)
        
        if dry_run: 
            print("Searching: ", sub_dir_path)
            print("Found first set: ", files1)
            print("Found second set: ", files1)

        if files1 and files2 and not dry_run:
            # Extract identified files
            file1 = files1[0]
            file2 = files2[0]
            
            # Prepare Output File
            output_dir = os.path.dirname(file1)
            output_file_with_subject = os.path.basename(sub_dir_path)+"_"+output_file_pattern
            
            # Prepare Backup Files
            copyfile1 = prepare_backup_files(file1)
            copyfile2 = prepare_backup_files(file2)
            
            # Re-register if neded
            if mni_template_path is not None:
                warp_nifti(copyfile=copyfile1, mni_template_path=mni_template_path)
                warp_nifti(copyfile=copyfile2, mni_template_path=mni_template_path)
            
            # Identify the downsampled file name
            downfile1 = copyfile1.split('.nii')[0]+"_mni152.nii"
            downfile2 = copyfile2.split('.nii')[0]+"_mni152.nii"
            
            # Add the aligned files
            downsample_image(copyfile1, downfile1)
            downsample_image(copyfile2, downfile2)
            
            try:
                importer = GiiNiiFileImport(os.path.dirname(downfile1), file_pattern="*copy_mni152.nii")
                imports = importer.run()
                imports["summated"] = imports.iloc[:,0] + imports.iloc[:,1]
                
                nifti_from_matrix(matrix=imports["summated"].to_numpy(), output_file=output_dir, output_name=output_file_with_subject, reference="MNI")
                
                # Clean up and remove the copy files. 
                os.remove(copyfile1)
                os.remove(copyfile2)
                os.remove(downfile1)
                os.remove(downfile2)
                
                print(f"\n ***Processed: {sub_dir_path}")
                
            except subprocess.CalledProcessError as e:
                print(f"Error processing {sub_dir_path}: {e}")
        elif dry_run:
            pass
        else:
            print(f"Missing file(s) in {sub_dir_path}")

Set MNI Template path to None unless you are doing fancy re-registrations. First attempt default

In [37]:
# User-defined variables
root_dir = r"/Volumes/One Touch/BWH_PD_DBS_MixedTarget/vtas"
file_pattern_1 = r"/*/*hemi-R.nii"
file_pattern_2 = r"/*/*hemi-L.nii"
output_file_pattern = r"sim-efield_model-simbio_hemi-bl.nii"

# Path to MNI template
mni_template_path =None

In [38]:
dry_run=False

In [39]:
merge_files(root_dir=root_dir, file_pattern_1=file_pattern_1, file_pattern_2=file_pattern_2, output_file_pattern=output_file_pattern, mni_template_path=mni_template_path, dry_run=dry_run)

  0%|          | 0/134 [00:00<?, ?it/s]

Attempting to import from: /Volumes/One Touch/BWH_PD_DBS_MixedTarget/vtas/sub-0037/stimulation/*copy_mni152.nii
        sub-CbctDbs0037_hemi-R_copy_mni152.nii  \
0                                          0.0   
1                                          0.0   
2                                          0.0   
3                                          0.0   
4                                          0.0   
...                                        ...   
902624                                     0.0   
902625                                     0.0   
902626                                     0.0   
902627                                     0.0   
902628                                     0.0   

        sub-CbctDbs0037_hemi-L_copy_mni152.nii  
0                                          0.0  
1                                          0.0  
2                                          0.0  
3                                          0.0  
4                                         

array([0., 0., 0., ..., 0., 0., 0.])

Image saved to: 
 /Volumes/One Touch/BWH_PD_DBS_MixedTarget/vtas/sub-0037/stimulation


  1%|          | 1/134 [00:02<05:56,  2.68s/it]

...


 ***Processed: /Volumes/One Touch/BWH_PD_DBS_MixedTarget/vtas/sub-0311
Attempting to import from: /Volumes/One Touch/BWH_PD_DBS_MixedTarget/vtas/sub-0316/stimulation/*copy_mni152.nii
        sub-CbctDbs0316_hemi-R_copy_mni152.nii  \
0                                          0.0   
1                                          0.0   
2                                          0.0   
3                                          0.0   
4                                          0.0   
...                                        ...   
902624                                     0.0   
902625                                     0.0   
902626                                     0.0   
902627                                     0.0   
902628                                     0.0   

        sub-CbctDbs0316_hemi-L_copy_mni152.nii  
0                                          0.0  
1                                          0.0  
2                                          0.0  
3                  

array([0., 0., 0., ..., 0., 0., 0.])

Image saved to: 
 /Volumes/One Touch/BWH_PD_DBS_MixedTarget/vtas/sub-0316/stimulation


100%|██████████| 134/134 [14:28<00:00,  6.48s/it]


 ***Processed: /Volumes/One Touch/BWH_PD_DBS_MixedTarget/vtas/sub-0316
